In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 50
BATCH_SIZE = 64

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size=BATCH_SIZE,
    shuffle=True)

0it [00:00, ?it/s]

9920512it [00:08, 1227989.03it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 37743.19it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:02, 777691.75it/s]                             
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


8192it [00:00, 21665.04it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [4]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p
    
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        
        # ADD dropout
        x = F.dropout(x,
                      training=self.training,
                      p=self.dropout_p
        )
        
        x = F.relu(self.fc2(x))
        
        # ADD dropout
        x = F.dropout(x,
                      training=self.training,
                      p=self.dropout_p
        )
        x = self.fc3(x)
        return x

In [5]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [7]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                        reduction='sum').item()
            
            # sum correct samples
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print("[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%".format(
        epoch, test_loss, test_accuracy))

[1] Test Loss: 0.5421, Accuracy: 82.70%
[2] Test Loss: 0.4135, Accuracy: 87.07%
[3] Test Loss: 0.3358, Accuracy: 89.42%
[4] Test Loss: 0.2856, Accuracy: 91.08%
[5] Test Loss: 0.2465, Accuracy: 92.16%
[6] Test Loss: 0.2207, Accuracy: 93.14%
[7] Test Loss: 0.1978, Accuracy: 93.87%
[8] Test Loss: 0.1834, Accuracy: 94.23%
[9] Test Loss: 0.1721, Accuracy: 94.72%
[10] Test Loss: 0.1658, Accuracy: 94.86%
[11] Test Loss: 0.1541, Accuracy: 95.17%
[12] Test Loss: 0.1495, Accuracy: 95.39%
[13] Test Loss: 0.1395, Accuracy: 95.65%
[14] Test Loss: 0.1375, Accuracy: 95.76%
[15] Test Loss: 0.1342, Accuracy: 95.77%
[16] Test Loss: 0.1301, Accuracy: 96.01%
[17] Test Loss: 0.1271, Accuracy: 95.99%
[18] Test Loss: 0.1233, Accuracy: 96.25%
[19] Test Loss: 0.1170, Accuracy: 96.34%
[20] Test Loss: 0.1163, Accuracy: 96.42%
[21] Test Loss: 0.1135, Accuracy: 96.51%
[22] Test Loss: 0.1113, Accuracy: 96.48%
[23] Test Loss: 0.1088, Accuracy: 96.57%
[24] Test Loss: 0.1075, Accuracy: 96.57%
[25] Test Loss: 0.1035, A

In [13]:
iter_tl = iter(train_loader)

In [14]:
i = next(iter_tl)

In [22]:
i[0].shape

torch.Size([64, 1, 28, 28])